In [ ]:
workDir = "/content/gdrive/MyDrive/"

In [ ]:
MOUNT_GDRIVE = True

In [ ]:
if MOUNT_GDIRVE:
  from google.colab import drive
  drive.mount("/content/gdrive")

Mounted at /content/drive


In [ ]:
import os
import re
import sys
import glob
import csv
import nltk
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from uuid import uuid4
from functools import reduce
from multiprocessing import Pool

In [ ]:
%cd /content/workDir

/content/drive/Shareddrives/NLP Bias


In [ ]:
!pip install wikiextractor

     |████████████████████████████████| 51kB 5.7MB/s 


Downloading a part of latest Wikipedia English Articles Dump

In [ ]:
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2

Extracting compressed file

In [ ]:
!wikiextractor -o "{workDir}/extracted" --processes 8 enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2

INFO: Preprocessing 'enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2' to collect template definitions: this may take some time.
INFO: Loaded 0 templates in 80.1s
INFO: Starting page extraction from enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2.
INFO: Using 8 extract processes.
INFO: Finished 8-process extraction of 27197 articles in 185.7s (146.5 art/s)


This code for post processing wikidump is inspired by the following gist
https://gist.github.com/snakers4/e0b0e68904db65671ca979639b337f7b

In [ ]:
def _remove_special_chars(string):
  # remove any special symbol
    return re.sub('[^a-zA-Z]', '', string)

def _trim_string(string):
    # remove extra spaces, remove trailing spaces, lower the case 
    return re.sub('\s+',' ',string).strip().lower()

def clean_string(string, min_len=2, max_len=30):
    string = _remove_special_chars(string)
    string = _trim_string(string)
    return string

def splitkeepsep(s, sep):
    cleaned = []
    s = re.split("(%s)" % re.escape(sep), s)

    for _ in s:
      #remove title
      if _.find('\n\n') != -1:
        _ = _[_.find('\n\n'):]
      #adding removed > tag to process together later
      if _!='' and _!=sep:
          cleaned.append(sep+'>'+_)
    return cleaned

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
def process_wiki_files(wiki_file):
    with open(wiki_file, encoding='utf-8') as f:
        content = f.read()

    articles = splitkeepsep(content,'<doc id=')
    df = pd.DataFrame(columns=['article_uuid','sentence','proc_sentence','proc_len'])

    for article in articles:
        uuid = uuid4()
        article = remove_html_tags(article).replace('\n','').replace(u'\xa0', u' ') 
      
        #remove empty articles
        if article == '':
          continue

        sentences = nltk.sent_tokenize(article)
        proc_sentences = [clean_string(sentence) for sentence in sentences]
        proc_lens = [len(sentence.split(' ')) for sentence in proc_sentences]
        
        temp_df = pd.DataFrame(
            {'article_uuid': [uuid]*len(sentences),
             'sentence': sentences,
             'proc_sentence':proc_sentences,
             'proc_len':proc_lens
            })
        df = df.append(temp_df)
    
    return df

In [ ]:
def list_multiprocessing(param_lst, func, **kwargs):
    workers = kwargs.pop('workers')

    with Pool(workers) as p:
        apply_lst = [([params], func, i, kwargs) for i,params in enumerate(param_lst)]
        result = list(tqdm(p.imap(_apply_lst, apply_lst), total=len(apply_lst)))

    result=sorted(result,key=lambda x:x[0])
    return [_[1] for _ in result]

def _apply_lst(args):
    params, func, num, kwargs = args
    return num, func(*params,**kwargs)

In [ ]:
nltk.download('punkt')

wiki_files = []

for f in glob.iglob(f'{workDir}/extracted/*/*', recursive=True):
    wiki_files.append(f)
  
df = list_multiprocessing(wiki_files, process_wiki_files, workers=4)

df = pd.concat(df).reset_index(drop=True)
df.article_uuid = df.article_uuid.astype(str)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving 5 sets with 1L sentences each from the parsed dataset

In [ ]:
for i in range(1, 6):
  df[i*100000:(i+1)*100000].to_csv(f'{workDir}/data/original/o{i}.csv')

The gender swap list has been curated from https://github.com/ranjita-naik/Gender_Swapping/tree/main/swaps and comprises of 260 pairs

In [ ]:
gender_dict = dict()

gender_words = []
with open(f'{workDir}/gender_list.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    gender_words.append((row[0], row[1]))
gender_words.pop(0)

In [ ]:
for m, f in gender_words:
  gender_dict[m] = f
  gender_dict[f] = m

In [ ]:
replaced = 0

In [ ]:
def replace_gender(sen, gender_dict):
  tok = sen.split(' ')
  tok2 = []
  global replaced
  for t in tok:
    if t in gender_dict:
      tok2.append(gender_dict[t])
      replaced += 1
    else:
      tok2.append(t)
  return " ".join(tok2)

Creating the swapped version of data

In [ ]:
for i in range(1, 6):
  with open(f'{workDir}/data/swapped/s{i}.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(list(df.columns.values))
    for row in df[i*100000:(i+1)*100000]:
      wr.writerow([row[0], row[1], row[2], replace_words(row[3], gender_dict), row[4]])

In [ ]:
print(replaced)